In [1]:
from threading import local
import datetime
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *


spark = SparkSession \
    .builder \
    .appName('Spark') \
    .master('local[*]') \
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', False).getOrCreate()
    


In [2]:
read_df = spark.read.csv(
    './adult/adult.csv', header=True, inferSchema=True)

print(read_df.columns)


['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'country', 'salary']


In [3]:
# windowSpec  = Window.partitionBy("country").orderBy("age")

rank_df = read_df.withColumn('new_date', dense_rank().over(Window.partitionBy('country').orderBy('age'))).show()

+---+-----------------+------+-------------+-------------+-------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+-------+------+--------+
|age|        workclass|fnlwgt|    education|education-num|     marital-status|        occupation|  relationship|               race|    sex|capital-gain|capital-loss|hours-per-week|country|salary|new_date|
+---+-----------------+------+-------------+-------------+-------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+-------+------+--------+
| 17|          Private|160118|         12th|            8|      Never-married|             Sales| Not-in-family|              White| Female|           0|           0|            10|      ?| <=50K|       1|
| 17|          Private|113301|         11th|            7|      Never-married| Handlers-cleaners|     Own-child|              White|   Male|           0|           0|          

In [4]:
sector_df = read_df.groupBy(read_df.education).count().show()

+-------------+-----+
|    education|count|
+-------------+-----+
|  Prof-school|  576|
|         10th|  933|
|      7th-8th|  646|
|      5th-6th|  333|
|   Assoc-acdm| 1067|
|    Assoc-voc| 1382|
|      Masters| 1723|
|         12th|  433|
|    Preschool|   51|
|          9th|  514|
|    Bachelors| 5355|
|    Doctorate|  413|
|      HS-grad|10501|
|         11th| 1175|
| Some-college| 7291|
|      1st-4th|  168|
+-------------+-----+

